In [ ]:
import itertools as it
from collections import Counter, defaultdict
from functools import partial
from glob import glob
from pathlib import Path

import gfapy
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.align as align
import paulssonlab.sequencing.cigar as scigar
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.gfa as sgfa
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
pl.enable_string_cache()

# Functions

In [ ]:
def concat_glob(filename):
    return pl.concat([pl.scan_ipc(f) for f in glob(filename)], how="diagonal")

In [ ]:
def label_columns(cols, func=None):
    expr = None
    for col in cols:
        if expr is None:
            expr = pl.when(pl.col(col).is_not_null())
        else:
            expr = expr.when(pl.col(col).is_not_null())
        if func is not None:
            lit = func(col)
        else:
            lit = col
        expr = expr.then(pl.lit(lit))
    return expr

In [ ]:
def load_sequencing(filename, filter=True):
    df = concat_glob(filename)
    df = df.with_columns(
        dup=pl.col("name").is_duplicated(),
        primary_alignment=pl.col("name").is_first_distinct(),
        e2e=pl.col("variants_path")
        .list.set_intersection(["<UNS9", ">UNS9", "<UNS3", ">UNS3"])
        .list.len()
        == 2,
        bc_e2e=pl.col("variants_path")
        .list.set_intersection(
            ["<BC:T7_prom", ">BC:T7_prom", "<BC:spacer2", ">BC:spacer2"]
        )
        .list.len()
        == 2,
    )
    if filter:
        df = df.filter(pl.col("primary_alignment"), pl.col("e2e"))
    return df

In [ ]:
def path_to_barcode_string(path_col, bits=list(range(30))):
    if isinstance(path_col, str):
        path_col = pl.col(path_col)
    return pl.concat_str(
        [
            pl.when(
                path_col.list.contains(f">BC:bit{bit}=1").or_(
                    path_col.list.contains(f"<BC:bit{bit}=1")
                )
            )
            .then(pl.lit("1"))
            .otherwise(pl.lit("0"))
            for bit in bits
        ]
    )

# 240703_pLIB476_bottlenecked

In [ ]:
%%time
df = load_sequencing(
    "/home/jqs1/scratch/sequencing/240703_pLIB476_bottlenecked/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()

In [ ]:
gfa = gfapy.Gfa.from_file(
    "/home/jqs1/scratch/sequencing/240703_pLIB476_bottlenecked/references/pLIB476jqs.gfa"
)

In [ ]:
df2 = processing.compute_divergences(
    df,
    list(dict.fromkeys(([s.split("=")[0] for s in gfa.segment_names]))),
    struct_name="extract_segments",
)

In [ ]:
df2 = df2.filter(pl.col("grouping_depth") != 1)

In [ ]:
df["extract_segments"].struct.fields

In [ ]:
df["max_divergence"].value_counts().sort("max_divergence")

In [ ]:
plt.hist(df4["max_divergence"], bins=100);

In [ ]:
df3.filter(pl.col("grouping_depth") < 10)

In [ ]:
df3.sort("grouping_depth").select(
    pl.col("grouping_depth"),
    frac_barcodes=pl.int_range(1, pl.len() + 1, dtype=pl.UInt32) / pl.len(),
    frac_reads=pl.col("grouping_depth").cum_sum() / pl.col("grouping_depth").sum(),
).to_pandas().hvplot.step("grouping_depth", logx=True, logy=False, where="pre")

In [ ]:
df4.filter(pl.col("max_divergence") > 0.05).sort("grouping_depth").select(
    pl.col("grouping_depth"),
    frac_barcodes=pl.int_range(1, pl.len() + 1, dtype=pl.UInt32) / pl.len(),
    frac_reads=pl.col("grouping_depth").cum_sum() / pl.col("grouping_depth").sum(),
).to_pandas().hvplot.step("grouping_depth", logx=True, logy=False, where="pre")

In [ ]:
%%time
df3.select(processing.categorical_list_hash(pl.col("grouping_path")).unique())

In [ ]:
df3["grouping_depth"].value_counts().sort("grouping_depth").with_columns(
    cum=pl.col("count").cum_sum(reverse=True)
).to_pandas().hvplot.step("grouping_depth", "cum", logy=True)

In [ ]:
plt.hist(df3["grouping_depth"], bins=1000, log=True);

# Joins

##  240703_pLIB476_bottlenecked with 240612_pLIB476_bottlenecked

In [ ]:
%%time
df_240703b = load_sequencing(
    "/home/jqs1/scratch/sequencing/240703_pLIB476_bottlenecked/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()
df_240610b = load_sequencing(
    "/home/jqs1/scratch/sequencing/240610_pLIB476_bottleneck/pLIB476_bottleneck/pLIB476/20240607_1433_MN35044_FAX60316_7d690112/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()

In [ ]:
%%time
cols = ["grouping_path", "variants_path", "grouping_depth"]
df_joined = df_240703b.select(cols).join(
    df_240610b.select(cols),
    on=processing.categorical_list_hash(pl.col("grouping_path")),
    how="full",
    suffix="_240610b",
)

In [ ]:
df_joined.group_by(
    pl.col("grouping_path").is_not_null(), pl.col("grouping_path_240610b").is_not_null()
).agg(pl.len()).rename(
    {
        "grouping_path": "240703_bottlenecked",
        "grouping_path_240610b": "240612_minion_bottlenecked",
    }
)

In [ ]:
df_240703b.sort("grouping_depth").select(
    pl.col("grouping_depth"),
    frac_barcodes=pl.int_range(1, pl.len() + 1, dtype=pl.UInt32) / pl.len(),
    frac_reads=pl.col("grouping_depth").cum_sum() / pl.col("grouping_depth").sum(),
).to_pandas().hvplot.step("grouping_depth", logx=True, logy=False, where="pre")

In [ ]:
df_240610b.sort("grouping_depth").select(
    pl.col("grouping_depth"),
    frac_barcodes=pl.int_range(1, pl.len() + 1, dtype=pl.UInt32) / pl.len(),
    frac_reads=pl.col("grouping_depth").cum_sum() / pl.col("grouping_depth").sum(),
).to_pandas().hvplot.step("grouping_depth", logx=True, logy=False, where="pre")

In [ ]:
df_joined.select("grouping_depth", "grouping_depth_240610b").to_pandas().hvplot.scatter(
    "grouping_depth", "grouping_depth_240610b"
)

##  240703_pLIB476_bottlenecked with 240510_pLIB476

In [ ]:
%%time
df_240703b = load_sequencing(
    "/home/jqs1/scratch/sequencing/240703_pLIB476_bottlenecked/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()
df_240513 = load_sequencing(
    "/home/jqs1/scratch/sequencing/240513_pLIB473_476/20240513_1645_2C_PAW46239_b49d575f/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()

In [ ]:
%%time
cols = ["grouping_path", "variants_path", "grouping_depth"]
df_joined = df_240703b.select(cols).join(
    df_240610b.select(cols),
    on=processing.categorical_list_hash(pl.col("grouping_path")),
    how="full",
    suffix="_240610b",
)

In [ ]:
df_joined.group_by(
    pl.col("grouping_path").is_not_null(), pl.col("grouping_path_240610b").is_not_null()
).agg(pl.len()).rename(
    {
        "grouping_path": "240703_bottlenecked",
        "grouping_path_240610b": "240612_minion_bottlenecked",
    }
)

## J23100

In [ ]:
align.pairwise_align(
    "GGAGTTTACGGCTAGCTCAGTCCTAGGTACAGTGCTAGC", "GGAGTTGACGGCTAGCTCAGTCCTAGGTACAGTGCTAGC"
)

In [ ]:
x = df2.filter(pl.col("promoter|seq") == "GGAGTTTACGGCTAGCTCAGTCCTAGGTACAGTGCTAGC")

In [ ]:
x[0, "cg"]

In [ ]:
x[0, "realign_cg"]

In [ ]:
idx = 0
path = x[idx, "variants_path"]
seq = x[idx, "consensus_seq"]

In [ ]:
name_to_seq = sgfa.gfa_name_mapping(gfa)
ref_seq = sgfa.assemble_seq_from_path(name_to_seq, path)

In [ ]:
seq

In [ ]:
processing.pairwise_align(
    seq,
    ref_seq,
    **{"cigar_as_string": True, "method": "parasail", "degenerate": True},
)

In [ ]:
processing.pairwise_align(
    seq,
    ref_seq,
    **{"cigar_as_string": True, "method": "parasail", "degenerate": False},
)

In [ ]:
x.columns

In [ ]:
y = processing.cut_cigar_df(
    x,
    gfa,
    path_column="variants_path",
    cigar_column="cg",
    sequence_column="consensus_seq",
    # phred_column="consensus_phred",
    query_start_column="query_start",
    query_end_column="query_end",
    query_length_column="query_length",
    path_start_column="path_start",
    path_end_column="path_end",
    struct_name="extract_segments2",
    keep_full=True,
)

In [ ]:
y2 = processing.cut_cigar_df(
    x,
    gfa,
    path_column="variants_path",
    cigar_column="realign_cg",
    sequence_column="consensus_seq",
    # phred_column="consensus_phred",
    # query_start_column="query_start",
    # query_end_column="query_end",
    # query_length_column="query_length",
    # path_start_column="path_start",
    # path_end_column="path_end",
    struct_name="extract_segments2",
    keep_full=True,
)

In [ ]:
plt.hist(
    x.select(pl.col("consensus_seq").str.len_bytes())["consensus_seq"],
    bins=100,
    log=True,
);

In [ ]:
len(x[0, "consensus_seq"])

In [ ]:
x.select("cg", "realign_cg")

In [ ]:
y2[0, "extract_segments2"]

In [ ]:
y[0, "extract_segments2"]

In [ ]:
x["consensus_seq"]

In [ ]:
x["cg"].value_counts(sort=True)

In [ ]:
x[2, "cg"]

In [ ]:
x[2].select(r"^.*\|cigar$")

In [ ]:
df["promoter|variant"].value_counts(sort=True)

In [ ]:
df2 = df.unnest("extract_segments")

In [ ]:
df2.filter(pl.col("promoter|variant") == "J23100", pl.col("promoter|divergence") == 0)[
    "promoter|seq"
].value_counts(sort=True).to_pandas()

In [ ]:
np.asarray(df2["ScmJ|divergence"]) != 0

In [ ]:
np.count_nonzero(np.asarray(df2["ScmJ|divergence"]))

In [ ]:
np.asarray(df2["ScmJ|divergence"])

In [ ]:
np.isnan(np.asarray(df2["ScmJ|divergence"]))

In [ ]:
def nonzero_frac(x):
    x = x[~np.isnan(x)]
    return np.count_nonzero(x) / len(x)


pl.DataFrame(
    [
        (c, nonzero_frac(np.asarray(df2[c])))
        for c in df2.columns
        if c.endswith("|divergence")
    ],
    schema=["col", "value"],
).sort("value", descending=True).to_pandas()

In [ ]:
df2["grouping_depth"].value_counts(sort=True)

In [ ]:
df2["pPhlF|seq"].value_counts(sort=True)

In [ ]:
df2["BetI|seq"].value_counts(sort=True)

In [ ]:
df2.filter(pl.col("grouping_depth") > 1)["BetI|seq"].value_counts(sort=True)

In [ ]:
with pl.Config(tbl_rows=20):
    display(
        df2.filter(pl.col("grouping_depth") > 1)["BetI|mismatches"].value_counts(
            sort=True
        )
    )

In [ ]:
seqs = df2["BetI|seq"].value_counts(sort=True)["BetI|seq"]
for idx in range(50):
    print(f">beti{idx}")
    print(seqs[idx])

In [ ]:
df2[0, "variants_path"].to_list()

In [ ]:
df2.group_by("promoter|variant").agg(
    pl.col("promoter|seq").value_counts(sort=True).struct[1].alias("counts")
).with_columns(
    total=pl.col("counts").list.sum(), correct=pl.col("counts").list.get(0)
).with_columns(
    frac_correct=pl.col("correct") / pl.col("total")
).to_pandas()

In [ ]:
df2.group_by("promoter|variant").agg(
    pl.col("CDS_mVenus|seq").value_counts(sort=True).struct[1].alias("counts")
).with_columns(
    total=pl.col("counts").list.sum(), correct=pl.col("counts").list.get(0)
).with_columns(
    frac_correct=pl.col("correct") / pl.col("total")
).to_pandas()

In [ ]:
df2["BCD_upstream|divergence"].value_counts(sort=True)

In [ ]:
df2["LacI|divergence"].value_counts(sort=True)

In [ ]:
df2["PhlF|seq"].value_counts(sort=True)

In [ ]:
df2["CDS_mVenus|divergence"].value_counts(sort=True)

In [ ]:
df2["Term_DT3|divergence"].value_counts(sort=True)

In [ ]:
df2["CDS_mVenus|divergence"].value_counts(sort=True)

In [ ]:
df2.filter(pl.col("promoter|variant") == "J23103")["promoter|mismatches"].value_counts(
    sort=True
).to_pandas()

In [ ]:
df2["CDS_mVenus|cigar"].value_counts(sort=True)

In [ ]:
df2["promoter|divergence"].value_counts(sort=True)

In [ ]:
df.filter(pl.col("promoter|variant") == "J23100")["promoter|seq"].value_counts(
    sort=True
).to_pandas()

In [ ]:
df.filter(pl.col("promoter|variant") == "J23100")["promoter|cigar"].value_counts(
    sort=True
).to_pandas()

In [ ]:
df.filter(pl.col("promoter|variant") == "J23100")["promoter|deletions"].value_counts(
    sort=True
)

In [ ]:
df.filter(pl.col("promoter|variant") == "J23100")["promoter|divergence"].value_counts(
    sort=True
)

In [ ]:
df2.sort("grouping_depth").select(
    pl.col("grouping_depth"),
    frac_barcodes=pl.int_range(1, pl.len() + 1, dtype=pl.UInt32) / pl.len(),
    frac_reads=pl.col("grouping_depth").cum_sum() / pl.col("grouping_depth").sum(),
).to_pandas().hvplot.step("grouping_depth", logx=True, logy=False, where="pre")

## Export to Eaton format

In [ ]:
%%time
df = load_sequencing(
    "/home/jqs1/scratch/sequencing/240703_pLIB476/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()

In [ ]:
gfa = gfapy.Gfa.from_file(
    "/home/jqs1/scratch/sequencing/sequencing_references/pLIB473-476.gfa"
)

In [ ]:
df = df.with_columns(
    pl.col("extract_segments").struct.with_fields(
        pl.field("repressor:promoter|variant").replace({"pLacI": "pTac"})
    )
)

In [ ]:
df = processing.compute_divergences(
    df,
    list(dict.fromkeys(([s.split("=")[0] for s in gfa.segment_names]))),
    struct_name="extract_segments",
)
df2 = df.unnest("extract_segments")

In [ ]:
df2["repressor:promoter|variant"].value_counts()

In [ ]:
df2["repressor|variant"].value_counts()

In [ ]:
df2.group_by("repressor:promoter|variant", "repressor|variant").agg(pl.len()).sort(
    "len", descending=True
)

In [ ]:
df2.select(pl.col(r"^.*\|divergence$")).quantile(0.9).to_pandas().T.sort_values(0)

In [ ]:
df2 = df2.with_columns(
    max_divergence=pl.max_horizontal(
        pl.col(r"^.*\|divergence$").exclude(
            "term:DT3|divergence", "JUNC10_UNS10|divergence"
        )
    )
)

In [ ]:
plt.hist(df2["max_divergence"], bins=100, log=True);

In [ ]:
df2.filter(pl.col("max_divergence") > 0.01)

In [ ]:
df2.filter(pl.col("max_divergence") > 0.01).select(
    pl.col(r"^.*\|divergence$")
).quantile(0.9).to_pandas().T.sort_values(0)

In [ ]:
plt.hist(df2["max_divergence"], bins=100, log=True);

In [ ]:
counts = df2.select(
    pl.struct("RBS1:RBS|seq", "RBS2:RBS|seq", "RBS3:RBS|seq").alias("foo")
)["foo"].value_counts(sort=True)

In [ ]:
counts["count"].to_pandas().hvplot.step(logy=True, logx=True)

In [ ]:
df["grouping_depth"].sort(descending=True).to_pandas().hvplot.step(logy=True)

In [ ]:
counts.filter(pl.col("count") == 1)

In [ ]:
counts.filter(pl.col("count") > 5)

In [ ]:
with pl.Config(tbl_rows=100):
    display(counts)

In [ ]:
perfect_segments = [
    "pBetI",
    "RBS1:upstream",
    "LacI",
    "pTac",
    "RBS2:upstream",
    "PhlF",
    "pPhlF",
    "RBS3:upstream",
    "BetI",
]

In [ ]:
df2.filter(*[pl.col(f"{s}|divergence") == 0 for s in perfect_segments]).select(
    pl.struct("RBS1:RBS|seq", "RBS2:RBS|seq", "RBS3:RBS|seq").alias("foo")
)["foo"].value_counts(sort=True)

In [ ]:
filter_columns = [
    "promoter|divergence",
    "ScmJ|divergence",
    "RBS:RiboJ|divergence",
    "RBS:BCD_leader|divergence",
    "repressor:promoter|divergence",
    "repressor|divergence",
    "mVenus|divergence",
]

In [ ]:
157_002 / len(df2)

In [ ]:
df2.filter(*[pl.col(c) == 0 for c in filter_columns])

In [ ]:
%%time
df_eaton = (
    df2.with_columns(
        barcode=path_to_barcode_string("variants_path"),
        reference=pl.lit(""),
        alignmentstart=1,
        cigar=pl.lit(""),
        subsample=pl.lit(""),
    )
    .rename({"consensus_seq": "consensus"})
    .select(
        "barcode",
        "consensus",
        "reference",
        "alignmentstart",
        "cigar",
        "subsample",
        "grouping_depth",
        "consensus_depth",
        "RBS|seq",
        "repressor:promoter|variant",
        "repressor|variant",
        "promoter|divergence",
        "ScmJ|divergence",
        "RBS:RiboJ|divergence",
        "RBS:BCD_leader|divergence",
        "repressor:promoter|divergence",
        "repressor|divergence",
        "mVenus|divergence",
    )
    .sort("barcode")
    .with_row_index(name="barcodeid")
    .with_row_index(name="")
)

In [ ]:
df_eaton.write_parquet("240718_pLIB473-475_eaton_export.parquet")

In [ ]:
df_eaton.write_csv("240718_pLIB473-475_eaton_export.tsv", separator="\t")

In [ ]:
!pwd